guesslang requires tensorflow 2.5 which does not run on apple silicon. Using colab to download and label the github dataset

In [1]:
!pip install guesslang datasets zstandard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 KB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 87.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import datasets
import guesslang
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
PYTHON_THRESHOLD = 0.9
LANG_THRESHOLD = 0.8
N_SEQS = 100_000
LANGS = {
    'C++',
    'C',
    'Go',
    'HTML',
    'Java',
    'JavaScript',
    'PHP',
    'Python',
    'XML'
}
#C++, C, GO, HTML, Java, Javascript, PHP, Python, XML

In [4]:
pile = datasets.load_dataset('the_pile', split="test", streaming=True)
github_pile = pile.filter(lambda x: x['meta']['pile_set_name'] == 'Github')
github_subset = github_pile.take(N_SEQS)
github_subset = datasets.Dataset.from_list(list(github_subset))

In [5]:
guesser = guesslang.Guess()
prob_list = []
lang_list = []
for i in range(len(github_subset)):
    probs = guesser.probabilities(github_subset[i]['text'])
    prob_list.append(probs[0][1])
    lang_list.append(probs[0][0] if probs[0][0] in LANGS else 'other')
    if i % 1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [6]:
github_subset = github_subset.add_column('lang_prob', prob_list)
github_subset = github_subset.add_column('lang', lang_list)

lang_id_dataset = github_subset.filter(lambda x: x['lang'] != 'other' and x['lang_prob'] >= LANG_THRESHOLD)
python_dataset = github_subset.filter(lambda x: x['lang'] == 'Python' and x['lang_prob'] >= PYTHON_THRESHOLD)

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

In [7]:
len(lang_id_dataset), len(python_dataset)

(7232, 607)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [9]:
data_path = '/content/gdrive/MyDrive/datasets/github/'

lang_id_dataset.save_to_disk(os.path.join(data_path, 'github_lang_id.hf'))
python_dataset.save_to_disk(os.path.join(data_path, 'python_source.hf'))

Flattening the indices:   0%|          | 0/8 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/7232 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/607 [00:00<?, ? examples/s]

In [10]:
import numpy as np

In [11]:
np.unique(np.array(lang_id_dataset['lang']), return_counts=True)

(array(['C', 'C++', 'Go', 'HTML', 'Java', 'JavaScript', 'PHP', 'Python',
        'XML'], dtype='<U10'),
 array([ 949, 1019,  817,  519, 1183,  716,  631,  637,  761]))